# Q&A RAG application using open-source models

In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "llama3:8b"


In [33]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

In [34]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 

In [35]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know". Please be precise and do not give any false information 
that does not occur in the context. You are only expected to answer from the context.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know". Please be precise and do not give any false information \nthat does not occur in the context. You are only expected to answer from the context.\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

ÖRNEK 1

In [36]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("KVK_07 KVK  Özel Nitelikli KVKK Politikası.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='UNIFREE DUTY FREE İŞLETMECİLİĞİ A.Ş.  \n \n \nÖZEL NİTELİKLİ KİŞİSEL VERİLERİN KORUNMASI POLİTİKASI  \n \n \n \n \nKVK_07', metadata={'source': 'KVK_07 KVK  Özel Nitelikli KVKK Politikası.pdf', 'page': 0}),
 Document(page_content='ÖZEL NİTELİKLİ KİŞİSEL \nVERİLERİN KORUNMASI \nPOLİTİKASI    Doküman No  KVK_07  \nYayın Tarihi  01.03.2023  \nRevizyon No   \nRevizyon Tarihi No   \n \n2 \n \n \nİÇİNDEKİLER  \n \n1. GİRİŞ  ................................ ................................ ................................ .................  3 \n2. ÖZEL NİTELİKLİ KİŞİSEL VERİLERİ NASIL İŞLİYORUZ?  ................................ .............  3 \n3. ÖZEL NİTELİKLİ KİŞİSEL VERİLERİ İŞLEME İLKELERİMİZ  ................................ ..........  4 \n4. ÖZEL NİTELİKLİ KİŞİSEL VERİLERİN AKTARILMASI  ................................ ..................  5 \n5. ÖZEL NİTELİKLİ KİŞİSEL VERİLERİNİZİ HANGİ HUKUKİ SEBEPLERLE İŞLİYORUZ?  ....... 6 \n6. ÖZEL NİTELİKLİ VERİLERİN İŞ

In [37]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [38]:
retriever = vectorstore.as_retriever()
retriever.invoke("Özel nitelikli kişisel veriler")

[Document(page_content='ÖZEL NİTELİKLİ KİŞİSEL \nVERİLERİN KORUNMASI \nPOLİTİKASI    Doküman No  KVK_07  \nYayın Tarihi  01.03.2023  \nRevizyon No   \nRevizyon Tarihi No   \n \n7 \n \n1. Özel nitelikli kişisel verilerin güvenliğine yönelik işbu Politika hazırlanmış ve yürürlüğe  \nalınmıştır.  \n \n2. Özel nitelikli kişisel verilerin işlenmesi süreçlerinde yer alan çalışanlara yönelik  olarak \naşağıdaki önlemler alınmıştır:  \n \n• Kanun ve buna bağlı yönetmelikler ile özel nitelikli kişisel veri güvenliği konularında \ndüzenli olarak eğitimler verilmektedir.  \n• Özel nitelikli kişisel veriye erişimi olan çalışanlar ile özel n itelikli kişisel verileri gizli \ntutacakları, hukuka aykırı bir şekilde kullanmayacakları ve korumak için en yüksek \nderecede özeni göstereceklerine ilişkin taahhütnameler imzalanmaktadır.  \n• Verilere erişim yetkisine sahip kullanıcıların, yet ki kapsamları ve süreleri net olarak \ntanımlanmaktadır.  \n• Bu çalışanların yetki kontrolleri  periyodik olarak g

In [39]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [40]:
questions = [
    "Şirket özel nitelikli kişisel verileri hangi hukuki sebeplerle işler?",
    "Şirket özel nitelikli verileri nasıl aktarır?",
    "Şirket özel nitelikli verileri ne kadar süre saklar ve sonra ne yapar?",
    "Şirket özel nitelikli veriler için hangi teknik ve idari tedbirleri alır?",
    "Şirket çalışanları için özel nitelikli veri işleme süreçlerinde hangi önlemleri alır?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Şirket özel nitelikli kişisel verileri hangi hukuki sebeplerle işler?
Answer: Şirket özel nitelikli kişisel verileri başta 6102 sayılı Türk Ticaret Kanunu, 6098 sayılı Türk Borçlar Kanunu olmak üzere KVKK m. 5’te düzenlenen hukuki sebepler çerçevesinde işler. Bu sebepler şunlardır: 
a) Kanunlarda açıkça öngörülmesi.
b) Fiili imkânsızlık nedeniyle rızasını açıklayamayacak durumda bulunan veya rızasına hukuki geçerlilik tanınmayan kişinin kendisinin ya da bir başkasının hayatı veya beden bütünlüğünün korunması için zorunlu olması.
c) Bir sözleşmenin kurulması veya ifasıyla doğrudan doğruya ilgili olması kaydıyla, sözleşmenin taraflarına ait kişisel verilerin işlenmesinin gerekli olması.
ç) Veri sorumlusunun hukuki yükümlülüğünü yerine getirebilmesi için zorunlu olması.
d) İlgili kişinin kendisi tarafından alenileştirilmiş olması.
e) Bir hakkın tesisi, kullanılması veya korunması için veri işlemenin zorunlu olması.
f) İlgili kişinin temel hak ve özgürlüklerine zarar vermemek kay

ÖRNEK 2

In [47]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("KVK_10 Elektronik Veri imha Prosedürü.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='UNIFREE DUTY FREE İŞLETMECİLİĞİ A.Ş.  \n \n \nELEKTRONİK VERİ İMHA PROSEDÜRÜ  \n \n \nKVK _10', metadata={'source': 'KVK_10 Elektronik Veri imha Prosedürü.pdf', 'page': 0}),
 Document(page_content='ELEKTRONİK VERİ İMHA \nPROSEDÜRÜ  Doküman No  KVK_10  \nYayın Tarihi  01.03.2023  \n Revizyon No   \nRevizyon Tarihi No   \n \n1 \n \n \n \nİÇİNDEKİLER  \n \n1. GİRİŞ  ................................ ................................ ................................ ................................ .........  2 \n2. KAPSAM  ................................ ................................ ................................ ................................ .... 2 \n3. HEDEF KULLANICI  ................................ ................................ ................................ ................  2 \n4. YÖNTEMİ  ................................ ................................ ................................ ................................ .. 2 \n5. SORUMLULUK  ....

In [48]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [49]:
retriever = vectorstore.as_retriever()
retriever.invoke("Veri İhlali")

[Document(page_content='ELEKTRONİK VERİ İMHA \nPROSEDÜRÜ  Doküman No  KVK_10  \nYayın Tarihi  01.03.2023  \n Revizyon No   \nRevizyon Tarihi No   \n \n4 \n \n6. KAYITLAR  \n \nVeri imha kayıtları çıktıları belirlenen klasörde dosyalanır.  \n7. REFERANSLAR  \n \nElektronik Veri İmha Formu', metadata={'source': 'KVK_10 Elektronik Veri imha Prosedürü.pdf', 'page': 4}),
 Document(page_content='ELEKTRONİK VERİ İMHA \nPROSEDÜRÜ  Doküman No  KVK_10  \nYayın Tarihi  01.03.2023  \n Revizyon No   \nRevizyon Tarihi No   \n \n3 \n \nYeniden yazılabilir ortamda (harici diskler, yeniden yazılabilir DVD ve CD) saklanan verinin \ngüvenli imhasına ilişkin yöntem, üzerine veri yazma yöntemidir.  \nYeniden yazma işlemi özel geliştirilmiş programların kullanılmasıyla yapılır. Güvenilir disk \nüzerine yazma yazılımının kullanılması verilerin yanlış ellere geçmesini engellemek için \ngüvenilir bir çözümdür.  \nSöz konusu program tüm harici disk üzerindeki bölümlerde, kısımlarda veya kullanılmamış \nkısımda 

In [50]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [51]:
questions = [
    "Veri ihlal şüphesi durumunda ilk adım nedir?",
    "Kişisel veri ihlalinin tanımı nedir?",
    "KVK Komitesi’nin sorumlulukları nelerdir?",
    "Veri ihlalinin bildirim süresi ne kadardır?",
    "Veri ihlali bildirimi hangi bilgilerle yapılır?, bu bilgileri açıkla",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Veri ihlal şüphesi durumunda ilk adım nedir?
Answer: İlk adım, veri ihlal şüphesi durumunda Elektronik Veri İmha Formu oluşturmak ve doldurmaktır.

Question: Kişisel veri ihlalinin tanımı nedir?
Answer: I don't know.

Question: KVK Komitesi’nin sorumlulukları nelerdir?
Answer: I don't know.

Question: Veri ihlalinin bildirim süresi ne kadardır?
Answer: I don't know.

Question: Veri ihlali bildirimi hangi bilgilerle yapılır?, bu bilgileri açıkla
Answer: I don't know.

